In [1]:
#import libraries
import csv
import pandas as pd
from geopy.geocoders import Nominatim

In [2]:
# read in data
data = pd.read_csv('addresses.csv')

In [4]:
#clean data
#data = data.rename(columns = {'Are you available for pickup on Friday, May 20th between 9am-12pm?':'20th am', 'Are you available for pickup on Friday, May 20th between 1-4pm?':'20th pm', 'Are you available for pickup on Saturday, May 21st between 9am-12pm?':'21st am', 'Are you available for pickup on Saturday, May 21st between 1-4pm?':'21st pm'}, axis=1)
                   #'Are you available for pickup on Friday, May 27th between 9am-12pm?':'27th am','Are you available for pickup on Friday, May 27th between 1-4pm?':'27th pm', 'Are you available for pickup on Saturday, May 28th between 9am-12pm?':'28th am', 'Are you available for pickup on Saturday, May 28th between 1-4pm?':'28th pm'}, inplace = True)
days = ['20th am', '20th pm', '21st am', '21st pm', '27th am', '27th pm', '28th am', '28th pm']
selected = ['Name', 'Address Line 1', 'Address Line 2', 'City','Zip Code']+ days


new_data = data[selected]


In [5]:
#make full address and select number of pickups
num_pickups = 88
new_data['address'] = new_data['Address Line 1'] + ', '+ new_data['City']
#new_data = new_data.drop([20, 32, 50], axis = 0)
new_data = new_data.iloc[:num_pickups]

/tmp/ipykernel_49/3008747992.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['address'] = new_data['Address Line 1'] + ', '+ new_data['City']


In [6]:
#convert dates
for row in days:
    new_data[row] = [1 if x == 'Yes' else 0 for x in new_data[row].values]
    

#create numeric indicator for availability
new_data['sum'] = new_data['20th am'] + new_data['20th pm'] + new_data['21st am'] + new_data['21st pm'] + new_data['27th am'] + new_data['27th pm'] + new_data['28th am'] + new_data['28th pm']


In [7]:
geolocator = Nominatim(user_agent="example app")

addresses = new_data['address'].values

In [8]:
coordinates = []

for i in range(0, num_pickups):
    location = geolocator.geocode(addresses[i])
    coordinates += [[location.point.latitude, location.point.longitude]]

#add coordinates column to dataframe

new_data['coordinates'] = coordinates

In [9]:
#start groups + add people with only one time slot
one = new_data[new_data['sum'] == 1]
data_no_one = new_data.drop(one.index, axis = 0)


first_fri_am = one[one['20th am'] == 1]
first_fri_pm = one[one['20th pm'] == 1]
first_sat_am = one[one['21st am'] == 1]
first_sat_pm = one[one['21st pm'] == 1]
sec_fri_am = one[one['27th am'] == 1]
sec_fri_pm = one[one['27th pm'] == 1]
sec_sat_am = one[one['28th am'] == 1]
sec_sat_pm = one[one['28th pm'] == 1]

tables = [first_fri_am, first_fri_pm, first_sat_am, first_sat_pm, sec_fri_am, sec_fri_pm, sec_sat_am, sec_sat_pm]
tables

#data_no_one

[             Name       Address Line 1 Address Line 2      City  Zip Code  \
 18    Atisha Amin    1888 Berkeley Way            415  Berkeley     94703   
 30   Jarle Volden  2149 Russell Street            NaN  Berkeley     94705   
 52  Olivier Masse   2322 Shattuck Ave.       Apt. 206  Berkeley     94704   
 57  Tiffany Villa   2614 Regent Street            NaN  Berkeley     94704   
 
     20th am  20th pm  21st am  21st pm  27th am  27th pm  28th am  28th pm  \
 18        1        0        0        0        0        0        0        0   
 30        1        0        0        0        0        0        0        0   
 52        1        0        0        0        0        0        0        0   
 57        1        0        0        0        0        0        0        0   
 
                           address  sum  \
 18    1888 Berkeley Way, Berkeley    1   
 30  2149 Russell Street, Berkeley    1   
 52   2322 Shattuck Ave., Berkeley    1   
 57   2614 Regent Street, Berkeley    1

In [10]:
google = 0
for i in data_no_one.index:
    availability = data_no_one.loc[i][days].values
    smallest = list()
    distances = list()
    item_coordinates = data_no_one.loc[i]['coordinates']
    for j in range(0,8):
        if availability[j] == 1:
            tbl_coordinates = tables[j]['coordinates'].values
            if len(tbl_coordinates) == 0:
                smallest += [0]
            else:
                distances = list()
                for k in range(0, len(tbl_coordinates)):
                    if len(tbl_coordinates) == 0:
                        smallest += [0]
                        break;
                    coords = tbl_coordinates[k]
                    x = (item_coordinates[0] - coords[0])**2
                    y = (item_coordinates[1] - coords[1])**2
                    distances += [((x + y)**0.5)]
                    google += 1
                if distances == []:
                    smallest += [0]
                smallest += [min(distances)]
        else:
            smallest += [1000]
    minimum_index = smallest.index(min(smallest))
    tables[minimum_index] = tables[minimum_index].append(data_no_one.loc[i])

In [11]:
google

1920

In [16]:
#reset index for each table

#for i in range(0, 8):
#    tables[i] = tables[i].sort_index()
#    display(tables[i])

In [17]:
for i in range(0,8):
    print(len(tables[i]))

33
4
17
2
2
5
10
15


In [18]:
# check length of each table and add indexes that could have some added and ones that could have some removed
remove = []
add = []

for i in range(0,8):
    length = len(tables[i])
    if length > 13:
        remove += [i]
    elif length < 13:
        add += [i]
        
remove

[0, 2, 7]

In [19]:
#rearrange lists
for item in remove:
    more_than_one = tables[item][tables[item]['sum'] > 1]
    more_than_one_index = more_than_one.index
    for index in reversed(more_than_one_index):
        smallest = list()
        distances = list()
        if len(tables[item]) <= 13:
            break;
        time_slots = more_than_one.loc[index][days]
        item_coordinates = more_than_one.loc[index]['coordinates']
        for j in range(0, 8):
            if time_slots[j] == 1 and add.count(j) > 0 and item != j:
                tbl_coordinates = tables[item]['coordinates'].values
                for k in range(0, len(tbl_coordinates)):
                    coords = tbl_coordinates[k]
                    x = (item_coordinates[0] - coords[0])**2
                    y = (item_coordinates[1] - coords[1])**2
                    distances += [((x + y)**0.5)]
                    google += 1
                smallest += [min(distances)]
            else:
                smallest += [1000]
        minimum_index = smallest.index(min(smallest))
        #check if this has to be iloc instead of loc -- changed to iloc
        tables[minimum_index] = tables[minimum_index].append(more_than_one.loc[index])
        tables[item] = tables[item].drop(index, axis = 0)
            

In [20]:
google

3134

In [21]:
#reset index for each table

#for i in range(0, 8):
#    tables[i] = tables[i].sort_index()
#    print(days[i])
#    print(len(tables[i]))
#    display(tables[i])

In [22]:
#validate table lengths
for i in range(0, 8):
    print(len(tables[i]))

13
26
13
5
2
6
10
13


In [42]:
#putting in order of distance
#address off of interstate 80 as the starting point since that's where we were told trucks would start
#change this if starting point changes 
geolocator = Nominatim(user_agent="example app")
starting = geolocator.geocode('650 University Avenue, Berkeley')
coords_starting = [starting.point.latitude, starting.point.longitude]

for i in range(0, 8):
    working_tbl = tables[i]
    new_tbl = pd.DataFrame()
    distances = []
    #get starting point
    for item in working_tbl.index:
        item_coordinates = working_tbl.loc[item]['coordinates']
        x = (item_coordinates[0] - coords_starting[0])**2
        y = (item_coordinates[1] - coords_starting[1])**2
        distances += [((x + y)**0.5)]
    minimum_index = distances.index(min(distances))
    new_tbl= new_tbl.append(working_tbl.iloc[minimum_index])
    distances = []
    
    #set new starting point to measure distance
    starting = working_tbl.iloc[minimum_index]['coordinates']
    
    print(working_tbl.index[minimum_index])
    display(working_tbl)
    to_drop = working_tbl.index[minimum_index]
    working_tbl = working_tbl.drop(to_drop, axis = 0)
    

    #order all other points
    while len(working_tbl) > 0:
        distances = []
        for item in working_tbl.index:
            item_coordinates = working_tbl.loc[item]['coordinates']
            x = (item_coordinates[0] - starting[0])**2
            y = (item_coordinates[1] - starting[1])**2
            distances += [((x + y)**0.5)]
        minimum_index = distances.index(min(distances))
        new_tbl= new_tbl.append(working_tbl.iloc[minimum_index])
        
        #set new starting point to measure distance
        starting = working_tbl.iloc[minimum_index]['coordinates']
        print(working_tbl.index[minimum_index])
        to_drop = working_tbl.index[minimum_index]
        working_tbl = working_tbl.drop(to_drop, axis = 0)
        
        

2


,Name,Address Line 1,Address Line 2,City,Zip Code,20th am,20th pm,21st am,21st pm,27th am,27th pm,28th am,28th pm,address,sum,coordinates
2,Morgan Foy,755 Ohlone Ave,NaN,Albany,94706,1,1,1,1,1,0,0,0,"755 Ohlone Ave, Albany",5,"[37.88550325103963, -122.30155449899276]"
6,Romeet Sen,30 Panoramic Way,NaN,Berkeley,94704,1,1,0,0,0,0,0,0,"30 Panoramic Way, Berkeley",2,"[37.867969328355684, -122.24822859104025]"
7,Kristine Kushner,1 Ardilla Road,(go 5 ft past ardilla to find our home),orinda,94563,1,1,0,0,1,1,0,0,"1 Ardilla Road, orinda",4,"[37.8893942, -122.1999919]"
11,Ruiting Wang,512 El Cerrito Plaza,"Metro510, S103",El Cerrito,94530,1,1,1,1,1,1,1,1,"512 El Cerrito Plaza, El Cerrito",8,"[37.899286700000005, -122.29700322066763]"
15,Vy Ngo,2828 College Ave,NaN,Berkeley,94705,1,1,1,1,0,0,0,0,"2828 College Ave, Berkeley",4,"[37.85923681773388, -122.25340614640984]"
18,Atisha Amin,1888 Berkeley Way,415,Berkeley,94703,1,0,0,0,0,0,0,0,"1888 Berkeley Way, Berkeley",1,"[37.8721946, -122.2738386]"
25,Michelle Kong,2537 Benvenue Ave,NaN,Berkeley,94015,1,1,0,0,0,0,0,0,"2537 Benvenue Ave, Berkeley",2,"[37.864116, -122.254981]"
27,Gisselle Alvarez,1934 Hearst Ave,NaN,Berkeley,94709,1,1,0,0,0,0,0,0,"1934 Hearst Ave, Berkeley",2,"[37.8731829, -122.27130704999999]"
30,Jarle Volden,2149 Russell Street,NaN,Berkeley,94705,1,0,0,0,0,0,0,0,"2149 Russell Street, Berkeley",1,"[37.85709468401329, -122.26458753080793]"
45,Didien Meyahnwi,2299 Piedmont avenue,NaN,Berkeley,94720,0,0,0,0,0,0,0,0,"2299 Piedmont avenue, Berkeley",0,"[37.86967705, -122.25166042209258]"


11
18
27
78
52
30
57
25
15
6
45
7
51


,Name,Address Line 1,Address Line 2,City,Zip Code,20th am,20th pm,21st am,21st pm,27th am,27th pm,28th am,28th pm,address,sum,coordinates
0,Lisa Ng,2150 Channing Way,NaN,Berkeley,94704,1,1,0,0,1,1,0,0,"2150 Channing Way, Berkeley",4,"[37.86577975, -122.26586777112328]"
13,Jarle Volden,2149 Russell Street,NaN,Berkeley,94705,0,1,1,0,0,0,0,0,"2149 Russell Street, Berkeley",2,"[37.85709468401329, -122.26458753080793]"
32,Vasudha Rao,1420 Alcatraz Ave,NaN,Berkeley,94531,1,1,1,1,0,0,0,0,"1420 Alcatraz Ave, Berkeley",4,"[37.847825163265306, -122.27919839795918]"
34,India Ott,2240 Blake St,NaN,Berkeley,94704,1,1,0,0,1,1,0,0,"2240 Blake St, Berkeley",4,"[37.86341293155966, -122.26409752963993]"
35,Elizabeth A Heckmann,5 panoramic way,NaN,Berkeley,94704,1,1,1,1,1,0,1,1,"5 panoramic way, Berkeley",7,"[37.8680260938155, -122.2483064867475]"
38,Drew Sherman,2435 Hilgard Avenue,Apt. #5,Berkeley,30309,1,1,0,0,0,0,0,0,"2435 Hilgard Avenue, Berkeley",2,"[37.87850362444464, -122.26218308216527]"
40,Simran Monder,2611 Hillegass Ave,NaN,Berkeley,94704,1,1,1,1,0,0,0,0,"2611 Hillegass Ave, Berkeley",4,"[37.86318292881721, -122.25621956995995]"
42,Roxanna Rabiee,2614 Regent Street,NaN,Berkeley,94704,1,1,1,0,0,0,0,0,"2614 Regent Street , Berkeley",3,"[37.86303871516606, -122.25772968563855]"
46,Mia Ndalugi,1930 Channing Way,Apt 1B,Berkeley,94704,1,1,0,0,0,0,0,0,"1930 Channing Way, Berkeley",2,"[37.865087700000004, -122.27072542721316]"
51,George Lambeth,1732 San Pablo Ave,Apart 6,Berkeley,94702,1,1,0,0,0,0,0,0,"1732 San Pablo Ave, Berkeley",2,"[37.872295163265306, -122.29300709183673]"


54
46
0
34
13
42
40
53
35
38
55
32
28


...

23
71
86
69


,Name,Address Line 1,Address Line 2,City,Zip Code,20th am,20th pm,21st am,21st pm,27th am,27th pm,28th am,28th pm,address,sum,coordinates
3,Noelle Link,2520 Hillegass Ave,Apt 318,Berkeley,94704,0,0,0,0,0,0,1,1,"2520 Hillegass Ave, Berkeley",2,"[37.864795, -122.256585]"
12,Sooyeon Lee,2299 Piedmont Avenue,Room 83T,Berkeley,94720,0,0,0,0,0,0,1,1,"2299 Piedmont Avenue, Berkeley",2,"[37.86967705, -122.25166042209258]"
19,Sooyeon Lee,2299 Piedmont Avenue,Room 83T,Berkeley,94720,0,0,0,0,0,0,1,1,"2299 Piedmont Avenue, Berkeley",2,"[37.86967705, -122.25166042209258]"
37,Lara Zargarian,2535 College,NaN,Berkeley,94704,0,0,0,0,1,0,1,0,"2535 College, Berkeley",2,"[37.86508364889443, -122.25378194152951]"
60,Nareauphol Liu,2423 Blake Street,NaN,Berkeley,94704,0,0,0,0,0,0,1,1,"2423 Blake Street, Berkeley",2,"[37.86418677789031, -122.25945964722908]"
61,Megan Chai,2467 Warring St,Apt 302,Berkeley,94704,0,0,0,0,1,1,1,1,"2467 Warring St, Berkeley",4,"[37.86665067591583, -122.25051104319019]"
67,Emma Yokota,2601 Channing Way,NaN,Berkeley,94704,0,0,0,0,1,1,1,1,"2601 Channing Way, Berkeley",4,"[37.86748285, -122.25629554881044]"
69,Megan Hu,2398 Parker St,Apt 11,Berkeley,94704,1,1,1,1,1,1,1,1,"2398 Parker St, Berkeley",8,"[37.862957, -122.260465]"
77,Isabel Li,2520 Hillegass Ave,Apt 103,Berkeley,94704,1,1,1,1,1,1,1,1,"2520 Hillegass Ave, Berkeley",8,"[37.864795, -122.256585]"
87,Anmol parande,2414 Parker Street,Apartment 5,Berkeley,94704,1,1,1,1,1,0,1,1,"2414 Parker Street, Berkeley",7,"[37.86287725, -122.2595496]"


87
60
3
77
67
37
61
12
19
58


,Name,Address Line 1,Address Line 2,City,Zip Code,20th am,20th pm,21st am,21st pm,27th am,27th pm,28th am,28th pm,address,sum,coordinates
4,Casey Chen,1806 Berkeley Way,APT 3,Berkeley,94703,0,0,0,0,1,1,1,1,"1806 Berkeley Way, Berkeley",4,"[37.87192145, -122.27486371192465]"
9,Chantel Johnson,2100 Channing Way,NaN,Berkeley,94704,0,0,0,0,0,0,1,1,"2100 Channing Way, Berkeley",2,"[37.86574, -122.267354]"
16,Cheyenne Smith,1919 Dwight Way,NaN,Berkeley,94704,0,0,0,0,1,1,1,1,"1919 Dwight Way, Berkeley",4,"[37.86350824170454, -122.27158698209672]"
24,Kaitlyn Kong,1540 Hearst Avenue,NaN,Berkeley,94703,0,0,0,0,1,1,1,1,"1540 Hearst Avenue, Berkeley",4,"[37.872059199999995, -122.28051644024188]"
26,Hankai ZHANG,1321 Spruce St,NaN,Berkeley,94709,0,0,0,0,0,0,1,1,"1321 Spruce St, Berkeley",2,"[37.88344378212882, -122.26620562142692]"
29,Daryl Yang,2018 Francisco Street,Apartment 302,Berkeley,94709,0,0,0,0,0,0,1,1,"2018 Francisco Street , Berkeley",2,"[37.87526215, -122.27022097740948]"
33,Ava Granados,5 Admiral Drive,Unit 303,Emeryville,94608,0,0,0,0,0,0,1,1,"5 Admiral Drive, Emeryville",2,"[37.8380851, -122.306446]"
36,Morgan Jennings,929 Oxford street,NaN,Berkeley,94707,0,0,0,0,1,1,1,1,"929 Oxford street , Berkeley",4,"[37.89148295, -122.26877606317348]"
39,Eric Zhang,6401 Shellmound St,NaN,Emeryville,94608,0,0,0,0,1,1,1,1,"6401 Shellmound St, Emeryville",4,"[37.84411, -122.293892]"
44,Samuel,2430 Martin Luther King Jr. Way,NaN,Berkeley,94704,1,1,1,1,1,1,1,1,"2430 Martin Luther King Jr. Way, Berkeley",8,"[37.864006599999996, -122.2724035682715]"


24
4
29
63
26
36
9
16
44
39
33
47


In [43]:
for i in range(0, 8):
    display(tables[i])

,Name,Address Line 1,Address Line 2,City,Zip Code,20th am,20th pm,21st am,21st pm,27th am,27th pm,28th am,28th pm,address,sum,coordinates
2,Morgan Foy,755 Ohlone Ave,NaN,Albany,94706,1,1,1,1,1,0,0,0,"755 Ohlone Ave, Albany",5,"[37.88550325103963, -122.30155449899276]"
6,Romeet Sen,30 Panoramic Way,NaN,Berkeley,94704,1,1,0,0,0,0,0,0,"30 Panoramic Way, Berkeley",2,"[37.867969328355684, -122.24822859104025]"
7,Kristine Kushner,1 Ardilla Road,(go 5 ft past ardilla to find our home),orinda,94563,1,1,0,0,1,1,0,0,"1 Ardilla Road, orinda",4,"[37.8893942, -122.1999919]"
11,Ruiting Wang,512 El Cerrito Plaza,"Metro510, S103",El Cerrito,94530,1,1,1,1,1,1,1,1,"512 El Cerrito Plaza, El Cerrito",8,"[37.899286700000005, -122.29700322066763]"
15,Vy Ngo,2828 College Ave,NaN,Berkeley,94705,1,1,1,1,0,0,0,0,"2828 College Ave, Berkeley",4,"[37.85923681773388, -122.25340614640984]"
18,Atisha Amin,1888 Berkeley Way,415,Berkeley,94703,1,0,0,0,0,0,0,0,"1888 Berkeley Way, Berkeley",1,"[37.8721946, -122.2738386]"
25,Michelle Kong,2537 Benvenue Ave,NaN,Berkeley,94015,1,1,0,0,0,0,0,0,"2537 Benvenue Ave, Berkeley",2,"[37.864116, -122.254981]"
27,Gisselle Alvarez,1934 Hearst Ave,NaN,Berkeley,94709,1,1,0,0,0,0,0,0,"1934 Hearst Ave, Berkeley",2,"[37.8731829, -122.27130704999999]"
30,Jarle Volden,2149 Russell Street,NaN,Berkeley,94705,1,0,0,0,0,0,0,0,"2149 Russell Street, Berkeley",1,"[37.85709468401329, -122.26458753080793]"
45,Didien Meyahnwi,2299 Piedmont avenue,NaN,Berkeley,94720,0,0,0,0,0,0,0,0,"2299 Piedmont avenue, Berkeley",0,"[37.86967705, -122.25166042209258]"


,Name,Address Line 1,Address Line 2,City,Zip Code,20th am,20th pm,21st am,21st pm,27th am,27th pm,28th am,28th pm,address,sum,coordinates
0,Lisa Ng,2150 Channing Way,NaN,Berkeley,94704,1,1,0,0,1,1,0,0,"2150 Channing Way, Berkeley",4,"[37.86577975, -122.26586777112328]"
13,Jarle Volden,2149 Russell Street,NaN,Berkeley,94705,0,1,1,0,0,0,0,0,"2149 Russell Street, Berkeley",2,"[37.85709468401329, -122.26458753080793]"
32,Vasudha Rao,1420 Alcatraz Ave,NaN,Berkeley,94531,1,1,1,1,0,0,0,0,"1420 Alcatraz Ave, Berkeley",4,"[37.847825163265306, -122.27919839795918]"
34,India Ott,2240 Blake St,NaN,Berkeley,94704,1,1,0,0,1,1,0,0,"2240 Blake St, Berkeley",4,"[37.86341293155966, -122.26409752963993]"
35,Elizabeth A Heckmann,5 panoramic way,NaN,Berkeley,94704,1,1,1,1,1,0,1,1,"5 panoramic way, Berkeley",7,"[37.8680260938155, -122.2483064867475]"
38,Drew Sherman,2435 Hilgard Avenue,Apt. #5,Berkeley,30309,1,1,0,0,0,0,0,0,"2435 Hilgard Avenue, Berkeley",2,"[37.87850362444464, -122.26218308216527]"
40,Simran Monder,2611 Hillegass Ave,NaN,Berkeley,94704,1,1,1,1,0,0,0,0,"2611 Hillegass Ave, Berkeley",4,"[37.86318292881721, -122.25621956995995]"
42,Roxanna Rabiee,2614 Regent Street,NaN,Berkeley,94704,1,1,1,0,0,0,0,0,"2614 Regent Street , Berkeley",3,"[37.86303871516606, -122.25772968563855]"
46,Mia Ndalugi,1930 Channing Way,Apt 1B,Berkeley,94704,1,1,0,0,0,0,0,0,"1930 Channing Way, Berkeley",2,"[37.865087700000004, -122.27072542721316]"
51,George Lambeth,1732 San Pablo Ave,Apart 6,Berkeley,94702,1,1,0,0,0,0,0,0,"1732 San Pablo Ave, Berkeley",2,"[37.872295163265306, -122.29300709183673]"


,Name,Address Line 1,Address Line 2,City,Zip Code,20th am,20th pm,21st am,21st pm,27th am,27th pm,28th am,28th pm,address,sum,coordinates
5,Kallan Bao,1950 Addison Street,NaN,BERKELEY,94704,0,0,1,1,0,0,0,0,"1950 Addison Street, BERKELEY",2,"[37.87063665966056, -122.27181069177556]"
8,Hao Xie,1715 Spruce Street,Apt 7,Berkeley,94709,1,1,1,1,0,0,0,0,"1715 Spruce Street, Berkeley",4,"[37.876691070757076, -122.26540837590929]"
17,Pauline Maninang,1517 Henry St.,NaN,Berkeley,94112,1,1,1,1,0,0,0,0,"1517 Henry St. , Berkeley",4,"[37.879566350000005, -122.270044227781]"
20,Kalea ibarra,2020 Channing way,Apt 14,Berkeley,94704,0,0,1,1,0,0,0,0,"2020 Channing way, Berkeley",2,"[37.86545427007576, -122.26960206584594]"
21,Atmika Ashok Pai,2131 Hearst Avenue,NaN,Berkeley,94709,1,1,1,1,0,0,0,0,"2131 Hearst Avenue, Berkeley",4,"[37.8742193, -122.26768715]"
22,Jose Vicente Tinoco Ochoa,1327 Milvia St,Apt 6,Berkeley,94709,0,0,1,1,0,0,1,1,"1327 Milvia St, Berkeley",4,"[37.88299459725338, -122.27191283173578]"
28,Joyce Zhu,1636 Martin Luther King Jr. Way,APT E,Berkeley,94709,1,1,1,0,0,0,0,0,"1636 Martin Luther King Jr. Way, Berkeley",3,"[37.87708520857746, -122.27378321795712]"
48,Miguel Ceja,2428 Ellsworth Street,Apt 105,Berkeley,94704,1,1,1,1,0,0,0,0,"2428 Ellsworth Street, Berkeley",4,"[37.865289795429376, -122.26326123353712]"
49,Ryan Tamura,1771 Highland Place,Apt 102,Berkeley,94709,0,0,1,0,0,0,0,0,"1771 Highland Place, Berkeley",1,"[37.87647, -122.255796]"
59,Chantel Nassi,2087 Delaware Street,Apt 5,Berkeley,94709,0,0,1,1,0,0,0,0,"2087 Delaware Street, Berkeley",2,"[37.8748577, -122.2689767]"


,Name,Address Line 1,Address Line 2,City,Zip Code,20th am,20th pm,21st am,21st pm,27th am,27th pm,28th am,28th pm,address,sum,coordinates
10,Shivika Dass,2732 Haste Street,,Berkeley,94704,0,0,0,1,0,0,0,0,"2732 Haste Street, Berkeley",1,"[37.86654596592581, -122.25335601549472]"
31,Frank Cuoco,2442 Piedmont Ave.,Apt. 16,Berkeley,94704,0,0,1,1,1,1,0,0,"2442 Piedmont Ave., Berkeley",4,"[37.86634695, -122.25206713922174]"
43,Victoria,865 Kinkead Way,Apt 304,Albany,94706,1,0,1,1,0,0,0,0,"865 Kinkead Way, Albany",3,"[37.88339965250313, -122.30231314636478]"
76,Carol Ng,1749 Oxford St.,Apt 4,Berkeley,94709,1,0,1,1,1,0,1,0,"1749 Oxford St. , Berkeley",5,"[37.875965300000004, -122.26627970000001]"
84,Catherine Gee,1929 Delaware St,Apt 1B,Berkeley,94709,0,0,1,1,1,1,1,1,"1929 Delaware St, Berkeley",6,"[37.87460725, -122.27200594286688]"
81,Kushagra Pandey,1950 Addison Street,Apt 728,Berkeley,94704,1,1,1,1,0,0,0,0,"1950 Addison Street, Berkeley",4,"[37.87063665966056, -122.27181069177556]"
80,Luke Cao,1799 Spruce Street,NaN,Berkeley,94709,1,1,1,1,0,0,0,0,"1799 Spruce Street, Berkeley",4,"[37.875184153568235, -122.26539196564998]"
79,Jake S. Lilian,2202 Blake Street,NaN,Berkeley,94704,1,1,1,1,0,0,0,0,"2202 Blake Street, Berkeley",4,"[37.86331134329434, -122.26485132603953]"
73,Aram Valartha Nayaki Subramanian,2423 Durant Avenue,NaN,Berkeley,94704,1,1,1,1,0,0,0,0,"2423 Durant Avenue, Berkeley",4,"[37.867722521809846, -122.26038638181502]"
72,Setareh Kianrokh,1260 Hopkins st,NaN,Berkeley,94702,1,1,1,1,1,1,1,1,"1260 Hopkins st, Berkeley",8,"[37.877435950000006, -122.28932821177501]"


,Name,Address Line 1,Address Line 2,City,Zip Code,20th am,20th pm,21st am,21st pm,27th am,27th pm,28th am,28th pm,address,sum,coordinates
1,Ben Wu,Durant 2400,NaN,Berkeley,94704,1,1,1,1,1,1,1,1,"Durant 2400, Berkeley",8,"[37.86745, -122.261139]"
14,Andrew Ferlo,1080 Jones St.,NaN,Berkeley,94710,0,0,0,0,1,1,0,0,"1080 Jones St., Berkeley",2,"[37.87596823939131, -122.29595586788311]"
41,Lauren Lee,2614 Regent St,NaN,Berkeley,94704,0,0,0,0,1,1,0,0,"2614 Regent St, Berkeley",2,"[37.86303871516606, -122.25772968563855]"
50,Joy Wang,2111 University Ave.,NaN,Berkeley,94704,0,0,0,0,1,1,0,0,"2111 University Ave., Berkeley",2,"[37.87212091462393, -122.26762214032055]"
74,Lois Koch,579 Grizzly Peak Blvd,NaN,Berkeley,94708,1,1,0,0,1,1,0,0,"579 Grizzly Peak Blvd, Berkeley",4,"[37.90159131852218, -122.26663352592796]"
66,Sarah Kardos,1950 rose street,Apt 3,Berkeley,94709,1,1,0,0,1,1,0,0,"1950 rose street , Berkeley",4,"[37.88150838461539, -122.27249984615385]"


,Name,Address Line 1,Address Line 2,City,Zip Code,20th am,20th pm,21st am,21st pm,27th am,27th pm,28th am,28th pm,address,sum,coordinates
23,Stephen Napolitano,1820 Euclid Ave.,NaN,Berkeley,94709,0,0,0,0,1,1,1,1,"1820 Euclid Ave. , Berkeley",4,"[37.875722494717095, -122.2604110348502]"
71,Justin Marc Alvarez,2548 Virginia Street,NaN,Berkeley,94709,0,0,0,0,0,1,0,0,"2548 Virginia Street, Berkeley",1,"[37.87777840394453, -122.25980967517397]"
75,Riley Clark,2222 haste st,NaN,Berkeley,94704,0,0,0,0,0,1,1,1,"2222 haste st, Berkeley",3,"[37.865199, -122.264516]"
86,Ash Reining,2503 Hilgard Ave,NaN,Berkeley,94709,0,0,0,0,1,1,1,1,"2503 Hilgard Ave, Berkeley",4,"[37.87869601717906, -122.2607774805951]"


,Name,Address Line 1,Address Line 2,City,Zip Code,20th am,20th pm,21st am,21st pm,27th am,27th pm,28th am,28th pm,address,sum,coordinates
3,Noelle Link,2520 Hillegass Ave,Apt 318,Berkeley,94704,0,0,0,0,0,0,1,1,"2520 Hillegass Ave, Berkeley",2,"[37.864795, -122.256585]"
12,Sooyeon Lee,2299 Piedmont Avenue,Room 83T,Berkeley,94720,0,0,0,0,0,0,1,1,"2299 Piedmont Avenue, Berkeley",2,"[37.86967705, -122.25166042209258]"
19,Sooyeon Lee,2299 Piedmont Avenue,Room 83T,Berkeley,94720,0,0,0,0,0,0,1,1,"2299 Piedmont Avenue, Berkeley",2,"[37.86967705, -122.25166042209258]"
37,Lara Zargarian,2535 College,NaN,Berkeley,94704,0,0,0,0,1,0,1,0,"2535 College, Berkeley",2,"[37.86508364889443, -122.25378194152951]"
60,Nareauphol Liu,2423 Blake Street,NaN,Berkeley,94704,0,0,0,0,0,0,1,1,"2423 Blake Street, Berkeley",2,"[37.86418677789031, -122.25945964722908]"
61,Megan Chai,2467 Warring St,Apt 302,Berkeley,94704,0,0,0,0,1,1,1,1,"2467 Warring St, Berkeley",4,"[37.86665067591583, -122.25051104319019]"
67,Emma Yokota,2601 Channing Way,NaN,Berkeley,94704,0,0,0,0,1,1,1,1,"2601 Channing Way, Berkeley",4,"[37.86748285, -122.25629554881044]"
69,Megan Hu,2398 Parker St,Apt 11,Berkeley,94704,1,1,1,1,1,1,1,1,"2398 Parker St, Berkeley",8,"[37.862957, -122.260465]"
77,Isabel Li,2520 Hillegass Ave,Apt 103,Berkeley,94704,1,1,1,1,1,1,1,1,"2520 Hillegass Ave, Berkeley",8,"[37.864795, -122.256585]"
87,Anmol parande,2414 Parker Street,Apartment 5,Berkeley,94704,1,1,1,1,1,0,1,1,"2414 Parker Street, Berkeley",7,"[37.86287725, -122.2595496]"


,Name,Address Line 1,Address Line 2,City,Zip Code,20th am,20th pm,21st am,21st pm,27th am,27th pm,28th am,28th pm,address,sum,coordinates
4,Casey Chen,1806 Berkeley Way,APT 3,Berkeley,94703,0,0,0,0,1,1,1,1,"1806 Berkeley Way, Berkeley",4,"[37.87192145, -122.27486371192465]"
9,Chantel Johnson,2100 Channing Way,NaN,Berkeley,94704,0,0,0,0,0,0,1,1,"2100 Channing Way, Berkeley",2,"[37.86574, -122.267354]"
16,Cheyenne Smith,1919 Dwight Way,NaN,Berkeley,94704,0,0,0,0,1,1,1,1,"1919 Dwight Way, Berkeley",4,"[37.86350824170454, -122.27158698209672]"
24,Kaitlyn Kong,1540 Hearst Avenue,NaN,Berkeley,94703,0,0,0,0,1,1,1,1,"1540 Hearst Avenue, Berkeley",4,"[37.872059199999995, -122.28051644024188]"
26,Hankai ZHANG,1321 Spruce St,NaN,Berkeley,94709,0,0,0,0,0,0,1,1,"1321 Spruce St, Berkeley",2,"[37.88344378212882, -122.26620562142692]"
29,Daryl Yang,2018 Francisco Street,Apartment 302,Berkeley,94709,0,0,0,0,0,0,1,1,"2018 Francisco Street , Berkeley",2,"[37.87526215, -122.27022097740948]"
33,Ava Granados,5 Admiral Drive,Unit 303,Emeryville,94608,0,0,0,0,0,0,1,1,"5 Admiral Drive, Emeryville",2,"[37.8380851, -122.306446]"
36,Morgan Jennings,929 Oxford street,NaN,Berkeley,94707,0,0,0,0,1,1,1,1,"929 Oxford street , Berkeley",4,"[37.89148295, -122.26877606317348]"
39,Eric Zhang,6401 Shellmound St,NaN,Emeryville,94608,0,0,0,0,1,1,1,1,"6401 Shellmound St, Emeryville",4,"[37.84411, -122.293892]"
44,Samuel,2430 Martin Luther King Jr. Way,NaN,Berkeley,94704,1,1,1,1,1,1,1,1,"2430 Martin Luther King Jr. Way, Berkeley",8,"[37.864006599999996, -122.2724035682715]"
